# The Determinants of Inflation

The objective of this section is to evaluate how the most relevant economic factors influence the abnormal shifts in Latin America over the last three years, although I do not try to assert which economic theory is more accurate. Then, I review the methods I use to measure their influence and dynamics over the 2020-2022 period.

## Preliminaries

First, it is necessary to import some libraries.

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

g = globals()

I'll use a `ggplot` theme for the graphs in this notebook.

In [2]:
!git clone https://gist.github.com/markusdumke/572e2a8546c54b322b48c0c792799b77 bw
theme_bw = './bw/theme_bw.mplstyle'
plt.style.use(theme_bw)

fatal: destination path 'bw' already exists and is not an empty directory.


I have the modified data frames, as created in the [Data Preparation](./00-data.ipynb) section.

In [9]:
countries = {'arg':'Argentina',
            'bra':'Brazil',
            'chl':'Chile',
            'col':'Colombia',
            'cri':'Costa Rica',
            'mex':'Mexico'
            }

vars = ['cpi','m2','mpr','uscpi','er','cons','ppi','ie']

for c in countries:
    g[f'dfm_{c}'] = pd.read_excel('./data/dfm.xlsx',
        sheet_name=c,
        index_col = 'date')

vars.remove('cpi')

## What Leads to Inflation?

From *only money matters* to *rational expectations*, there exists a great variety of macroeconomic arguments[^totonchi] around the causes of inflation in both the short and long run. Table [2](tab-variables) provides a summary of the variables I used.

[^totonchi]: See {cite}`totonchi2011`.

Modern economies have central banks that can directly set the quantity of currency in circulation. Moreover, it seems to be that money growth and the rate of increase in prices move together over long periods of time[^gls], being the excessive money growth over output growth one of the most adopted thesis on the causes of inflation in the long run. The first monetary policy variable I use, which are denoted by `MP` in Table [2](tab-variables), is money supply in the form of a three-year change. In the same way, modern central banks raise the official (nominal) interest rate when high inflation affect the economy. In this manner, I assume a Neo-Fisherianism viewpoint, in which the real interest rate is independent of monetary factors, and so the nominal interest rates and inflation move in the same direction, and MPRs are used as instruments to affect the price levels.

[^gls]: See {cite}`gls2020`.

The second group of variables corresponds to international factors `(IF)`, which take into account United States' inflation as well as nominal exchange rates. The first indicator is important due to the strong economic relationships between Latin American countries and the US, and because the 2020-2022 period was characterized by worldwide increases in prices. The national exchange rate to $\$1$ is implemented based on pass-through theory, which argues that a country's rate of inflation is more robustly affected by exchange rate fluctuations when the country widely depends on imports.

Private consumption is taken into account due to demand-pull theory `(DP)`; that is, increases in aggregate demand that generate inflation pressures due to this being larger than the aggregate supply at full employment levels. In contrast, cost-push theory `(CP)` proposes that rises in the cost of production commodities impulse inflation rates, hence the Producer Price Index is included. Finally, inflation expectations `(IE)` fit in the analysis in behalf of the Rational Expectations revolution of 1970s, which recognises that economic agents generate expectations based on past and current information, and take into account these expectations when making their decisions.

```{table} Theories and Variables
:name: tab-variables

| Theory                 | Variable | Data                                | Measure       |
|------------------------|----------|-------------------------------------|---------------|
| Monetary Policy        | m2       | M2                                  | 3y change     |
|                        | mpr      | Policy-Related Interest Rates       | 1y difference |
| International Factors  | usacpi   | Consumer Price Index                | 1y change     |
|                        | er       | Nominal Exchange Rate to 1          | 1y change     |
| Demand Pull            | cons     | Household's Consumption Expenditure | 1y change     |
| Cost Push              | ppi      | Producer Prices Index               | 1y change     |
| Inflation Expectations | ie       | Inflation Expectations              | 1y difference |
```

## Influence Method

Subsequently, an attribution methodology based on the Mahalanobis distance will be used to examine the influence of the variables that were presented before. The Mahalanobis distance[^mahalanobis] is a measure of divergence between groups in terms of multiple characteristics[^mclachlan]. Another way to introduce it is with variability: due to the positive or negative relation that multivariate data could show, the Mahalanobis approach assigns a larger distance to those observations that are not only further away from their means, as Euclidean distances would do, but to points from the sample that show a larger variability. The application used in this paper is based on {cite}`kinlaw2022`, in which the authors compute this distance to analyse inflationary regimes in the United States. Let the Mahalanobis distance for country $c$ at time $t$ from regime $r$ be denoted as $\delta_{c,t,r}$:

$$
\delta_{c,t,r}=
    \left(x_{c,t}-\mu_{c,r}\right)^{T}
    S_{c,r}^{-1}
    \left(x_{c,t}-\mu_{c,r}\right)
$$ (eq-mah)

[^mahalanobis]: Developed in {cite}`mahalanobis1936`.
[^mclachlan]: See {cite}`mclachlan1999`.

In [10]:
n_states = 4
for c in countries:
    for r in range(n_states):
        #Variance-covariance matrices and mean vectors
        if g[f'dfm_{c}'][g[f'dfm_{c}']['predicted']==r].empty == False:
            g[f'S_{c}_{r}'] = pd.DataFrame(np.cov(g[f'dfm_{c}'][g[f'dfm_{c}']['predicted']==r][vars].T),
                                            columns=vars,
                                            index=vars)

            g[f'μ_{c}_{r}'] = g[f'dfm_{c}'][g[f'dfm_{c}']['predicted']==r][vars].mean()

        else:
            g[f'S_{c}_{r}'] = pd.DataFrame(np.cov(g[f'dfm_{c}'][vars].T),
                                            columns=vars,
                                            index=vars)

            g[f'μ_{c}_{r}'] = g[f'dfm_{c}'][vars].mean()

        for t in range(len(g[f'dfm_{c}'])):
            if np.linalg.det(g[f'S_{c}_{r}']) != 0:
                g[f'ẟ_{c}_{t}_{r}'] = (g[f'dfm_{c}'][vars].iloc[t]-g[f'μ_{c}_{r}']
                                )@np.linalg.inv(g[f'S_{c}_{r}']
                                )@(g[f'dfm_{c}'][vars].iloc[t]-g[f'μ_{c}_{r}'])

            else:
                g[f'ẟ_{c}_{t}_{r}'] = (g[f'dfm_{c}'][vars].iloc[t]-g[f'μ_{c}_{r}']
                                )@np.linalg.inv(np.cov(g[f'dfm_{c}'][vars].T)
                                )@(g[f'dfm_{c}'][vars].iloc[t]-g[f'μ_{c}_{r}'])

Where, for a specific country $c$, $x_{c,t}$ is a vector that contains the observations of the variables at time $t$; $\mu_{c,r}$ a vector that has the means of these variables for a certain regime $r$; and, $S^{-1}_{c,r}$ the inverse of the symmetric and positive semi-definite variance-covariance matrix of variables in regime $r$. The superscript $T$ the transpose of a matrix, and the result is a scalar. Next, the distance of Equation [3](eq-mah) is converted into a statistical likelihood according to a normal distribution, as follows:

$$
   L_{c,t,r} = 
    \frac{1}{\sqrt{det\left(2\pi S_{c,r}\right)}}
    exp \left( \frac{-\delta_{c,t,r}}{2}\right)
$$ (eq-likelihood)

In [11]:
for c in countries:
    for r in range(n_states):
        for t in range(len(g[f'dfm_{c}'])):
            if np.linalg.det(2*np.pi*g[f'S_{c}_{r}']) <= 0 or np.isinf(np.exp(-g[f'ẟ_{c}_{t}_{r}']/2)) == True:
                g[f'L_{c}_{t}_{r}'] = 0
            else:
                g[f'L_{c}_{t}_{r}'] = (1/np.sqrt(np.linalg.det(2*np.pi*g[f'S_{c}_{r}']))
                                    )*np.exp((-g[f'ẟ_{c}_{t}_{r}']/2))

/var/folders/rt/qhl8_0ld1rs4tqkb8syqwn7h0000gn/T/ipykernel_40752/1095699689.py:4: RuntimeWarning: overflow encountered in exp
  if np.linalg.det(2*np.pi*g[f'S_{c}_{r}']) <= 0 or np.isinf(np.exp(-g[f'ẟ_{c}_{t}_{r}']/2)) == True:


In Equation [4](eq-likelihood), $S_{c,r}$ is the variance-covariance matrix of the variables of country $c$ in regime $r$. Afterwards, the likelihood has to be rescaled to be interpreted as a probability. Let the probability of country $c$ at time $t$ to be in regime $r$ be denoted as $\rho_{c,t,r}$:

$$
  \rho_{c,t,r}=\frac{L_{c,t,r}}{\sum_{\text{all regimes r}}} L_{c,t,r}
$$ (eq-rho)

In [12]:
for c in countries:
    for r in range(n_states):
        for t in range(len(g[f'dfm_{c}'])):
            g[f'ρ_{c}_{t}_{r}'] = g[f'L_{c}_{t}_{r}'] / sum(g[f'L_{c}_{t}_{j}'] for j in range(n_states))

Next, in order to determine the importance of the variables I compute the derivative of the composite function[^chainrule] of Equation [5](eq-rho) with respect to the variables vector $x_{c,t}$. For a specific regime $i$, this derivative corresponds to:

$$
\frac{\partial \rho_{c,t,i}}{\partial x_{c,t}}=
    \rho_{c,t,i}
    \left[
    \left(
    \sum_{\text{all regimes r}}\rho_{c,t,r}
    \frac{\partial \delta_{c,t,r}}{\partial x_{c,t}}
    \right)
    -\frac{\partial \delta_{c,t,i}}{\partial x_{c,t}}
    \right]
$$ (eq-derivative)

[^chainrule]: Using the chain rule: $\dfrac{\partial \rho_{c,t,r}}{\partial L_{c,t,r}} \dfrac{\partial L_{c,t,r}}{\partial \delta_{c,t,r}} \dfrac{\partial \delta_{c,t,r}}{\partial x_{c,t}}$

In [13]:
for c in countries:
    for r in range(n_states):
        for t in range(len(g[f'dfm_{c}'])):
            if np.linalg.det(g[f'S_{c}_{r}']) != 0:
                g[f'dx_{c}_{t}_{r}'] = np.linalg.inv(g[f'S_{c}_{r}'])@(
                    (g[f'dfm_{c}'][vars].iloc[t]-g[f'μ_{c}_{r}'])
                    )
            else: 
                g[f'dx_{c}_{t}_{r}'] = np.linalg.inv(np.cov(g[f'dfm_{c}'][vars].T))@(
                    (g[f'dfm_{c}'][vars].iloc[t]-g[f'μ_{c}_{r}'])
                    )
            
            g[f'dρdx_{c}_{t}_{r}'] = g[f'ρ_{c}_{t}_{r}']*(
                    sum( g[f'ρ_{c}_{t}_{r}']*g[f'dx_{c}_{t}_{r}'] for j in range(n_states))
                    -g[f'dx_{c}_{t}_{r}']
                    )

Equation [6](eq-derivative) is a derivative with respect to the variables vector, meaning that the output is also a vector and it represents the sensitivity of regime $r$ to the variables of the model. Having four regimes, and with the objective of obtaining the total sensitivity of the variables, I take an average[^avg] across regimes:

$$
    \zeta_{c,t}=
    \frac{1}{4}
    \sum_{\text{all regimes r}}
    \left\lvert
    \frac{\partial \rho_{c,t,r}}{\partial x_{c,t}}
    \right\rvert
$$ (eq-sens)

[^avg]: Because of Equation [5](eq-rho), the average will sum to one. Hence, the average of the absolute values is computed.

In [14]:
for c in countries:
    for t in range(len(g[f'dfm_{c}'])):
        g[f'𝛇_{c}_{t}'] = (1/n_states)*sum(abs(g[f'dρdx_{c}_{t}_{j}']) for j in range(n_states))

In here, $\zeta_{c,t}$ denotes the total sensitivity vector of the variables of country $c$ at a specific point in time $t$. Finally, rescaling Equation [7](eq-sens) with the standard deviations of the full sample allows to obtain the relative importance vector of country $c$ at a given point in time $t$:

$$
    \psi_{c,t} = \frac{\zeta_{c,t}\sigma_{c}}
    {\sum_{\text{all variables v}}\left\lvert
    \zeta_{c,t}\sigma_{c}\right\rvert}
$$ (eq-totalsens)

In [15]:
for c in countries:
    g[f'ir_{c}'] = pd.DataFrame()
    for t in range(len(g[f'dfm_{c}'])):
        g[f'𝜓_{c}_{t}'] = g[f'𝛇_{c}_{t}']*np.std(g[f'dfm_{c}'][vars]) / (
            sum(g[f'𝛇_{c}_{t}']*np.std(g[f'dfm_{c}'][vars]))
            )

        g[f'ir_{c}'][t] = g[f'𝜓_{c}_{t}']

    g[f'ir_{c}']=(g[f'ir_{c}']*100).T.set_index(g[f'dfm_{c}'].index)

In [16]:
with pd.ExcelWriter('./data/ir.xlsx', engine='xlsxwriter') as writer:
    for c in countries:
        g[f'ir_{c}'].to_excel(writer, sheet_name=c)